In [17]:
import numpy as np

In [18]:
img = np.arange(16).reshape((4, 4)) + 1
img

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16]])

In [19]:
def box_sum_bad(img: np.ndarray, k: int) -> np.ndarray:
    cum_cols = np.cumsum(img, axis=0)
    ii = np.cumsum(cum_cols, axis=1)
    ii = np.pad(ii, ((1, 0), (1, 0)), mode="constant")
    strides = np.lib.stride_tricks.as_strided(
        ii,
        shape=(ii.shape[0] - k, ii.shape[1] - k, k + 1, k + 1),
        strides=ii.strides + ii.strides
    ).astype(np.int64)
    elements_for_summing = strides[:, :, [0, k]][:, :, :, [0, k]]
    elements_for_summing[:, :, 0, 1] = -elements_for_summing[:, :, 0, 1]
    elements_for_summing[:, :, 1, 0] = -elements_for_summing[:, :, 1, 0]
    result = np.sum(elements_for_summing, axis=(-2, -1))

    return result


def box_sum(img: np.ndarray, k: int) -> np.ndarray:
    ii = img.astype(np.int64).cumsum(axis=0).cumsum(axis=1)
    ii = np.pad(ii, ((1, 0), (1, 0)), mode="constant")

    return ii[k:, k:] - ii[:-k, k:] - ii[k:, :-k] + ii[:-k, :-k]

In [20]:
k = 3
sum = box_sum(img, k)
sum

array([[54, 63],
       [90, 99]])

In [21]:
def box_mean_valid(img: np.ndarray, k: int) -> np.ndarray:
    sums = box_sum(img, k)
    means = sums / (k ** 2)

    return means


In [22]:
box_mean_valid(img, k)

array([[ 6.,  7.],
       [10., 11.]])

In [23]:
def box_mean_same(img: np.ndarray, k: int) -> np.ndarray:
    if k < 1 or k % 2 == 0:
        raise ValueError("k must be a positive odd integer")

    p = k // 2
    img_p = np.pad(img, ((p, p), (p, p)), mode="constant", constant_values=0)
    sums = box_sum(img_p, k)

    return np.round(sums / (k * k), 2)

In [24]:
box_mean_same(img, k)

array([[ 1.56,  2.67,  3.33,  2.44],
       [ 3.67,  6.  ,  7.  ,  5.  ],
       [ 6.33, 10.  , 11.  ,  7.67],
       [ 5.11,  8.  ,  8.67,  6.  ]])